### Fake News Classifier:

datasets:https://www.kaggle.com/c/fake-news/data
lstm Reading:https://colah.github.io/posts/2015-08-Understanding-LSTMs/
wordembedding: https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:
## Drop Nan Values
df = df.dropna()

In [4]:
## Get the indeendent features
X = df.drop('label',axis=1)

In [5]:
y = df['label']

In [6]:
X.shape

(18285, 4)

In [7]:
y.shape

(18285,)

In [8]:
import tensorflow as tf

In [9]:
tf.__version__

'2.1.0'

In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout

In [11]:
## voc_size set
voc_size = 5000 

### One hot Representation

In [12]:
X

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [13]:
message = X.copy()

In [14]:
message['title']

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                        Why the Truth Might Get You Fired
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799                            What Keeps the F-35 Alive
Name: title, Length: 18285, dtype: object

In [15]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [16]:
message.reset_index(inplace=True)

In [18]:
!pip install nltk

  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434680 sha256=de8e6017c24f138cf5e05abdff3100082e1db0a81f861b95c345d73627215d35
  Stored in directory: c:\users\my pc\appdata\local\pip\cache\wheels\45\6c\46\a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk


In [19]:
import nltk
import re
from nltk.corpus import stopwords

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\MY
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
## Data Preprocessing

from nltk.stem.porter import PorterStemmer
pos = PorterStemmer()
corpus =[]

for i in range(0,len(message)):
    
    review = re.sub('[^a-zA-Z]',' ',message['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [pos.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [22]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [23]:
# one_hot representation
one_hotrpr = [one_hot(words,voc_size)for words in corpus]
one_hotrpr

[[3244, 1363, 2122, 2353, 3140, 4454, 1203, 2497, 3600, 3679],
 [2166, 3563, 648, 4343, 3953, 2345, 4025],
 [3629, 3598, 2510, 4398],
 [2477, 292, 348, 4425, 2693, 2696],
 [145, 3953, 1609, 4210, 305, 3685, 3953, 2652, 4688, 1191],
 [1901,
  1519,
  1047,
  4449,
  4096,
  948,
  843,
  1678,
  4990,
  4817,
  4189,
  4724,
  1201,
  4412,
  4025],
 [145, 4721, 4187, 4560, 629, 1673, 123, 656, 3395, 1753, 1816],
 [3727, 1989, 399, 1562, 3831, 1514, 948, 159, 3395, 1753, 1816],
 [2099, 1072, 1945, 1452, 4251, 1523, 950, 3600, 948, 90],
 [2888, 2760, 224, 4616, 3134, 96, 987, 621],
 [4863, 2356, 113, 4123, 2386, 47, 618, 3831, 1281, 775, 3668],
 [4425, 3563, 3140, 1523, 948, 3831],
 [3659, 4322, 2704, 3171, 267, 4676, 2886, 102, 123],
 [3379, 2377, 3429, 91, 2289, 1017, 2843, 3395, 1753, 1816],
 [77, 312, 2851, 2345, 1301, 3395, 1753, 1816],
 [107, 4416, 4098, 2877, 472, 1874, 2005, 4962, 4542, 2925],
 [4076, 3514, 3563],
 [213, 3513, 2898, 778, 948, 2943, 248, 4025],
 [3082, 2346, 648, 

### Embedding Representation:

In [24]:
sent_length=20

embedded_docs=pad_sequences(one_hotrpr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2497 3600 3679]
 [   0    0    0 ... 3953 2345 4025]
 [   0    0    0 ... 3598 2510 4398]
 ...
 [   0    0    0 ... 3395 1753 1816]
 [   0    0    0 ... 1268 4925 4631]
 [   0    0    0 ... 2328 2600 4703]]


In [25]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3244,
       1363, 2122, 2353, 3140, 4454, 1203, 2497, 3600, 3679])

In [26]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.30))
model.add(LSTM(100))
model.add(Dropout(0.50))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [28]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [30]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 20s 2ms/sample - loss: 0.2905 - accuracy: 0.8676 - val_loss: 0.1979 - val_accuracy: 0.9132
Epoch 2/10
12250/12250 [==============================] - 15s 1ms/sample - loss: 0.1429 - accuracy: 0.9426 - val_loss: 0.2149 - val_accuracy: 0.9163
Epoch 3/10
12250/12250 [==============================] - 15s 1ms/sample - loss: 0.1021 - accuracy: 0.9628 - val_loss: 0.2516 - val_accuracy: 0.9158
Epoch 4/10
12250/12250 [==============================] - 15s 1ms/sample - loss: 0.0669 - accuracy: 0.9765 - val_loss: 0.2787 - val_accuracy: 0.9125
Epoch 5/10
12250/12250 [==============================] - 15s 1ms/sample - loss: 0.0476 - accuracy: 0.9826 - val_loss: 0.3236 - val_accuracy: 0.9142
Epoch 6/10
12250/12250 [==============================] - 16s 1ms/sample - loss: 0.0334 - accuracy: 0.9891 - val_loss: 0.3439 - val_accuracy: 0.9100
Epoch 7/10
12250/12250 [==============================] -

### Performance Metrics and Accuracy

In [31]:
y_pred=model.predict_classes(X_test)

In [32]:
from sklearn.metrics import confusion_matrix

In [33]:
confusion_matrix(y_test,y_pred)

array([[3138,  281],
       [ 299, 2317]], dtype=int64)

In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.903893951946976

### Prediction Accuracy 90% :